In [8]:
#Create Model

import boto3
from time import gmtime, strftime
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()
sess = sagemaker.Session()

training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")

sage = boto3.Session().client(service_name='sagemaker') 

model_name="Lewis-full-image-classification-model"  
print(model_name)

model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)

hosting_image = '811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:latest'

primary_container = {
    'Image': hosting_image,
    'ModelDataUrl': model_data,
}

create_model_response = sage.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])


'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


Lewis-full-image-classification-model
s3://planthealth/My-trial/output/My-trial-2020-07-21-12-12-18/output/model.tar.gz
arn:aws:sagemaker:us-east-1:622247710393:model/lewis-full-image-classification-model


In [17]:
#Create Endpoint Configuration with Amazon Elastic Inference
from time import gmtime, strftime


endpoint_config_name = 'testendpoint' + '-epc' 
endpoint_config_response = sage.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'AcceleratorType': 'ml.eia1.large',
        'VariantName':'AllTraffic'}])

print('Endpoint configuration name: {}'.format(endpoint_config_name))
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))


Endpoint configuration name: testendpoint-epc
Endpoint configuration arn:  arn:aws:sagemaker:us-east-1:622247710393:endpoint-config/testendpoint-epc


In [22]:
%%time
import time
sagemaker = boto3.client('sagemaker')

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_name = 'testingendpoint'
print('Endpoint name: {}'.format(endpoint_name))

endpoint_params = {
    'EndpointName': endpoint_name,
    'EndpointConfigName': endpoint_config_name,
}
endpoint_response = sagemaker.create_endpoint(**endpoint_params)
print('EndpointArn = {}'.format(endpoint_response['EndpointArn']))

Endpoint name: testingendpoint
EndpointArn = arn:aws:sagemaker:us-east-1:622247710393:endpoint/testingendpoint
CPU times: user 19.7 ms, sys: 0 ns, total: 19.7 ms
Wall time: 232 ms
